# The EP full-text library - Lesson 2
This notebook expands on lesson 1 to dive into more advanced concepts of EPAB, the implementation in TIP of the EP full-text library. We will introduce querying by full text fields, divisionals and parents, and search report fields. As we did in the first notebook, we first create an instance of the EPAB library. Remember that by default we are getting access to a test database

In [ ]:
# Importing the EPAB client
from epo.tipdata.epab import EPABClient

# creating an instance of the EPAB client with the production database
epab = EPABClient(env='PROD')


## Querying by full text fields
Much like the [EP full-text search](https://www.epo.org/en/searching-for-patents/technical/ep-full-text), one of the most powerful features of the EPAB library is that it gives you access to the description, claims, title and abstract of the publications within the EPAB database. 

### Querying by the title
You can search for applications containing one or more terms in the title. When performing a first search for patent publications of a given technological concept, it is generally a good approach to search in the title, since when a publication contains the search term in the title it is likely that it is a good match for your search query. If you followed lesson 1, you probably can guess nomenclature of the search method: `query_title`.

In [ ]:
# querying by the title of the publication with the word 'covid'
q = epab.query_title('covid')
q.get_results("title", limit=5, output_type='list')


#### Understanding fulltext languages
You can see in the result that the title field contains a dictionary with three titles. It is very important, when working with fulltext, to take into consideration that the EPO publishes the fulltext fields in the three official languages: German, English, and French.

When you search for a term in a fulltext field, by default you will search in all three languages. This can be problematic. A good example of a search query that would yield different results in English and German is the word "Gift."

In English, "gift" refers to a present or something given willingly to someone without payment. However, in German, "Gift" means "poison." You can change this by specifying one or more of the official languages with the strings `EN`, `DE` and `FR`.

In [ ]:
# searching for publications with the word GIFT only in the English title
q = epab.query_title('gift', language="EN")
q.get_results("title", limit=5, )

#### Refresher of query combination
We saw in lesson 1 that we can combine queries to create more complex queries. Let's see if there are any publications that contain the word gift in both the German and English titles. 

In [ ]:
# we get a second query with publications mentioning poison, in German
r = epab.query_title('gift', language="DE")
print (f'publications with the word Gift in German', r)

#combining the two queries
s = q & r

print (f'Poisionus gifts found:', s)

### Case sensitivity
You have seen that we are querying in lowercase and the titles are displayed in all uppercase. It will come at no surprise that the search for full text terms is by default case insensitive. This can be overriden with `ignore_case=False`. Below we perform two queries with and without this parameter, to see the different results we get. 

In [ ]:
# searching for publications with the word GIFT only in the English title ignoring case
q = epab.query_title('gift', language="EN")
print (f'Publications with the word gift in any combination of lower and upper case', q)

q.get_results('title', limit=5)




In [ ]:
# searching for publications with the word GIFT only in the English title forcing lowercase
r = epab.query_title('gift', language="EN", ignore_case=False)
print (f'Publications with the word gift in lowercase', r)

r.get_results('title', limit=5)

### Multiple search terms
We can enter multiple search terms in the queries we run on EPAB by full text fields. When we enter multiple terms, by default these terms are combined with an `OR`

In [ ]:
# Searching a set of possible terms (e.g. synonyms)
q = epab.query_title("covid, corona virus, coronavirus", language="EN")
print (q)
q.get_results("title.en", output_type="list", limit=10)

#### Multiple search terms combined with AND
We can also query with several strings, and specify that they all should be present, with the `match_all` parameter.

In [ ]:
# We can also look for having multiple terms in the same title
q = epab.query_title("coronavirus, vaccine", match_all=True, language="EN")
print(q)
q.get_results("title.en", limit=5)

#### Multiple search terms with advanced combinations
What if you want to mix `AND` with `OR` with the combinations of terms? Combining queries comes in handy for this case. 

In [ ]:
# searching for synonims of Covid 
q = epab.query_title(search_terms="covid, corona virus, coronavirus", language="EN")

# searching for synonims of vaccine
r = epab.query_title(search_terms="vaccine%, inmun%", language="EN")

s = q & r

s.get_results('title.en', limit = 10)

### Querying abstract, claims and description
You can query other parts of the fulltext such as the claims, the abstract, and the description with the same methods, obviously changing the part of the fulltext in the method nomenclature. 

In [ ]:
# abstract search
q = epab.query_abstract("handover, base station", match_all=True, ignore_case=True)
print(q)
q.get_results("abstract", output_type="list", limit=2)

## Retrieving statistics from a query
Sometimes you will want to get statistics over the results of a query, before further processing it. The method `get_stats` returns a dataframe with the statistics over one or more selected fields. when you run this method on a query object, for the selected field(s) you will get the following information. 

- the `count` column reports the total number of occurrences of the corresponding field(s) value
- the `unique_publications` column reports the number of unique publications having that value
- the last two lines of the table are used to report the remainder and the total

### Statistics on patents about wireless communication networks
Let's look at an example. We will make a query for publications in the field of wireless communication networks, grouped in the CPC under H04W

In [12]:
# Running a query for all publications with CPC symbols starting with H04W
q = epab.query_ipc("H04W%")
q

183817 publications

In [14]:
# We want to see the distribution of the countries where the inventors mentioned in the publications resulting from the query live
q.get_stats("inventor.country")

,inventor.country,count,unique_publications
0,US,168276.0,51631.0
1,CN,113949.0,44220.0
2,KR,57087.0,16489.0
3,JP,47794.0,19227.0
4,SE,41752.0,16104.0
...,...,...,...
97,EC,1.0,1.0
98,GL,1.0,1.0
99,NC,1.0,1.0
100,KW,1.0,1.0


You can see that there are more inventors than publications. This happens because typically one application lists more than one inventor. We can also see what applicants are most active in the field of wireless communication networks

In [16]:
# We want to see the distribution of the countries where the inventors mentioned in the publications resulting from the query live
q.get_stats("applicant.name")

,applicant.name,count,unique_publications
0,"Huawei Technologies Co., Ltd.",17041.0,17041.0
1,Telefonaktiebolaget LM Ericsson (publ),10865.0,10864.0
2,"Samsung Electronics Co., Ltd.",8795.0,8795.0
3,Qualcomm Incorporated,8404.0,8404.0
4,ZTE Corporation,5884.0,5884.0
...,...,...,...
8840,"Pham, Thien, Van",1.0,1.0
8841,"Bhalla, Rajesh",1.0,1.0
8842,KATALYXER S.r.l.,1.0,1.0
8843,Eko Devices Inc.,1.0,1.0


Again remember that a patent application can name more than one applicant, so it is possible that the sum of the `count` field will be higher than the sum of the `unique_publications` field.